In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd 
import h3
import os
import pandas as pd
import math
from shapely.geometry import Point

from sklearn.neighbors import NearestNeighbors
from shapely.geometry import Polygon
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_absolute_error

import dask.dataframe as dd

from dask.diagnostics import ProgressBar

pd.set_option('display.max_columns', 500)



In [ ]:

import pandas as pd
from sqlalchemy import create_engine
#   Драйвер подключения к БД
engine = create_engine('')
sql = '''


'''
df = pd.read_sql(sql, con=engine)


In [ ]:
df = df[df['area_price'] > 500]
df['link_id'] = df['link'].apply(lambda x: x.split('_')[-1])
df['timestamp'] = pd.to_datetime(df['date_id'])
df = df.loc[df.groupby('link_id')['timestamp'].idxmax()]
def get_first_word(text):
  """Возвращает первое слово из строки."""
  words = text.split()
  if words:
    return words[0]
  else:
    return ''  # Возвращаем пустую строку, если текст пустой

# Создаем новую колонку 'first_word'
df['class'] = df['title'].apply(get_first_word)
df = df[df['class'] == 'Дом']


unique_dates = df['timestamp'].unique()
unique_dates.sort()

test_dates = unique_dates[-1:]

test_df = pd.DataFrame()
test_df = df[df['timestamp'].isin(test_dates)]

lists_of_dates = unique_dates[:-1]

df = df[df['timestamp'].isin(lists_of_dates)]

df['link_id'] = df['link'].apply(lambda x: x.split('_')[-1])
df['timestamp'] = pd.to_datetime(df['date_id'])
df = df.loc[df.groupby('link_id')['timestamp'].idxmax()]
unique_dates = df['timestamp'].unique()
unique_dates.sort()

lists_of_dates = unique_dates[:-1]

df = df[df['timestamp'].isin(lists_of_dates)]

dates = pd.to_datetime(lists_of_dates)

# Нахождение минимальной даты
min_date = dates.min()
# Вычитание минимальной даты из всех остальных и преобразование в дни
days_list = (dates - min_date).days
days_list = days_list.tolist()


dataframes = []
list_of_dfs = []
for i in range(len(df['date_id'].unique())):
    df_name = 'df' +'_'+ str(days_list[i])
    list_of_dfs.append(df_name)
    df_filtered = df[df['date_id'] == sorted(df['date_id'].unique())[i]]
    #dataframes[df_name] = df[df['date_of_parsing'] == df['date_of_parsing'].unique()[i]]
    dataframes.append(df_filtered)


# удаляем выбросы (если стоимость за кв.м. рассматриваемого участка меньше в 6 раз или больше в 10 отсноительно 10 ближайших участков, участок удаляется)
n_neighbors = 10 
resolution = 7
log_base = 2
min_neigbors_count_sum = 4
def drop_bad_values(df, n_neighbors):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    nn_model = NearestNeighbors(n_neighbors=n_neighbors)
    nn_model.fit(gdf[['longitude', 'latitude']])
    def calculate_averages(row):
        _, indices = nn_model.kneighbors([[row['longitude'], row['latitude']]])
        neighbors = gdf.iloc[indices[0][1:], :]
        area_price_mean = neighbors['area_price'].median()
        return pd.Series({'area_price_mean': area_price_mean})

    averages = df.apply(calculate_averages, axis=1)
    df[['area_mean']] = averages
    df['realative_difference'] = df['area_price']/df['area_mean']
    df = df[df['realative_difference'] >= 0.15]
    df = df[df['realative_difference'] < 10]
    df = df.drop(columns = ['realative_difference','area_mean'])
    return df 

list_of_izs = [drop_bad_values(df, n_neighbors) for df in dataframes]
concatenated_df = pd.concat(dataframes, ignore_index=True)
concatenated_df = concatenated_df.drop(columns=['area_mean','realative_difference'])

In [ ]:
# удаляем выбросы (если стоимость за кв.м. рассматриваемого участка меньше в 6 раз или больше в 10 отсноительно 10 ближайших участков, участок удаляется)
n_neighbors = 10 
resolution = 7
log_base = 2
min_neigbors_count_sum = 4
def drop_bad_values(df, n_neighbors):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    nn_model = NearestNeighbors(n_neighbors=n_neighbors)
    nn_model.fit(gdf[['longitude', 'latitude']])
    def calculate_averages(row):
        _, indices = nn_model.kneighbors([[row['longitude'], row['latitude']]])
        neighbors = gdf.iloc[indices[0][1:], :]
        area_price_mean = neighbors['area'].median()

        return pd.Series({'area_mean': area_price_mean})

    averages = df.apply(calculate_averages, axis=1)

    df[['area_mean']] = averages
    df['realative_difference'] = df['area']/df['area_mean']
    df = df[df['realative_difference'] >= 0.15]
    df = df[df['realative_difference'] < 10]
    df = df.drop(columns = ['realative_difference','area'])
    return df 

df_new = drop_bad_values(concatenated_df, n_neighbors) 



In [ ]:
def h3_latlng_to_cell(lat, lng, resolution):
    cell = h3.geo_to_h3(lat, lng, resolution)
    return cell

def from_cooordinates_to_hex(df, resolution):
    df['hex_7'] = df.apply(lambda row: h3.geo_to_h3(row['latitude'], row['longitude'], resolution), axis=1)
    return df

geo_df_izs = from_cooordinates_to_hex(concatenated_df, resolution)

geo_df_izs['count'] = geo_df_izs['hex_7'].map(geo_df_izs['hex_7'].value_counts())

geo_df_izs['q1_area_price'], geo_df_izs['q3_area_price'] = geo_df_izs['area_price'], geo_df_izs['area_price']

In [ ]:
#df_grouped = geo_df_izs.groupby('hex_7').agg({'area_price': 'median', 'count': 'max', }).reset_index()
df_grouped = geo_df_izs.groupby('hex_7').agg(
    area = ('area', 'median'),
    area_price = ('area_price', 'median'),
    count = ('count', 'max'),
    q3_area_price = ('area_price', lambda x: x.quantile(0.75)),
    q1_area_price = ('area_price', lambda x: x.quantile(0.25))
).reset_index()

df_grouped['mean_area_price'] = df_grouped['area_price']
df_grouped = df_grouped.drop(columns = 'area_price')

In [ ]:
ddf_grouped = dd.from_pandas(df_grouped, npartitions=10)  

In [ ]:
def calculate_mean_price_with_h3_neighbors(df, log_base, min_neigbors_count_sum):
    df['log_price'] = df['mean_area_price'] * (df['count'].apply(lambda x: math.log(x, log_base)))
    df['log_area'] = df['area'] * (df['count'].apply(lambda x: math.log(x, log_base)))
    df['log_q1_price'] = df['q1_area_price'] * (df['count'].apply(lambda x: math.log(x, log_base)))
    df['log_q3_price'] = df['q3_area_price'] * (df['count'].apply(lambda x: math.log(x, log_base)))
    df['log_count'] = (df['count'].apply(lambda x: math.log(x, log_base)))

    def calculate_for_row(row):
        h3_index = row['hex_7']
        try:
            neighbors_indices = h3.k_ring(h3_index, 1)
        except ValueError:
            print(f"Invalid H3 index: {h3_index}")
            return pd.Series([None, None, None, None, None, None])
        neighbors_indices = h3.k_ring(h3_index, 1)
        neighbors_indices = h3.k_ring(h3_index, 1)
        neighbors_indices = [neighbor for neighbor in neighbors_indices if neighbor != h3_index]
        neighbors_df = df[df['hex_7'].isin(neighbors_indices)]
        summary_count = neighbors_df['count'].sum() + row['count']

        if neighbors_df.empty or summary_count < min_neigbors_count_sum:
            return pd.Series([row['mean_area_price'], 0, 0, row['q1_area_price'], row['q3_area_price'], summary_count])

        mean_price = (row['mean_area_price']*row['count'] + neighbors_df['log_price'].sum())/(row['count']+neighbors_df['log_count'].sum())
        q1_price = (row['q1_area_price']*row['count'] + neighbors_df['log_q1_price'].sum())/(row['count']+neighbors_df['log_count'].sum())
        q3_price = (row['q3_area_price']*row['count'] + neighbors_df['log_q3_price'].sum())/(row['count']+neighbors_df['log_count'].sum())
        return pd.Series([mean_price, q1_price, q3_price, row['q1_area_price'], row['q3_area_price'], summary_count])

    result = df.apply(calculate_for_row, axis=1)
    df[['mean_price_neighbors', 'q1_price', 'q3_price', 'q1_area_price_new', 'q3_area_price_new', 'summary_count_current_plus_neighbors']] = result
    return df

def get_mean_price(ddf, log_base, min_neigbors_count_sum):
    ddf = ddf.map_partitions(
        calculate_mean_price_with_h3_neighbors, 
        log_base=log_base, 
        min_neigbors_count_sum=min_neigbors_count_sum
    )
    return ddf

# Пример использования (предполагая, что у вас есть Dask DataFrame `ddf_grouped`)
ddf_grouped = get_mean_price(ddf_grouped, log_base, min_neigbors_count_sum)

# Вычисление результата (если нужно)
ddf_grouped = ddf_grouped.compute()

ddf_grouped = ddf_grouped[ddf_grouped['summary_count_current_plus_neighbors'] > 3]
ddf_grouped

In [ ]:
def find_neighbors_with_h3(gdf, hex_column='hex_7', k=1):
  """
  Находит ближайшие гексы для каждого гекса в GeoDataFrame с помощью H3 k_ring.

  Args:
    gdf: GeoDataFrame с данными.
    hex_column: Название столбца с номерами гексов.
    k: Расстояние (в шагах) для поиска соседей.

  Returns:
    Множество с ID всех соседних гексов.
  """
  neighbor_ids = set()

  for hex_id in gdf[hex_column]:
    # Находим соседей с помощью h3.k_ring
    neighbors = h3.k_ring(hex_id, k)
    neighbor_ids.update(neighbors)

  return neighbor_ids

neighbor_ids = find_neighbors_with_h3(ddf_grouped)

def add_missing_hexes(df, all_neighbor_ids, hex_column='hex_7'):

  missing_hexes = set(df[hex_column]) - all_neighbor_ids

  if missing_hexes:
    # Создаем новую строку с пустыми значениями
    new_row = pd.Series({col: None for col in df.columns})
    new_row[hex_column] = list(missing_hexes)  # Добавляем ID отсутствующих гексов

    # Добавляем новую строку в DataFrame
    df = df.append(new_row, ignore_index=True)

  return df

def add_missing_hexes_from_set(df, hex_set, hex_column='hex_7'):

  existing_hexes = set(df[hex_column])
  missing_hexes = hex_set - existing_hexes

  if missing_hexes:
    # Создаем DataFrame с отсутствующими гексами
    new_df = pd.DataFrame({hex_column: list(missing_hexes)})

    # Добавляем другие столбцы с пустыми значениями
    for col in df.columns:
      if col != hex_column:
        new_df[col] = None
  
    # Объединяем DataFrame с исх одным
    df = pd.concat([df, new_df], ignore_index=True)

  return df

df_with_missing = add_missing_hexes_from_set(ddf_grouped, neighbor_ids)


In [ ]:
def calculate_mean_price_with_h3_neighbors(df, log_base, min_neigbors_count_sum, df_all):
    def calculate_for_row(row):
        h3_index = row['hex_7']
        try:
            neighbors_indices = h3.k_ring(h3_index, 1)
        except ValueError:
            print(f"Invalid H3 index: {h3_index}")
            return pd.Series([None, None, None, None, None, None])
        
        # Удаляем текущий индекс из списка соседей
        neighbors_indices = [neighbor for neighbor in neighbors_indices if neighbor != h3_index]
        
        # Получаем данные по соседям
        neighbors_df = df_all[df_all['hex_7'].isin(neighbors_indices)]
        
        # Суммируем count соседей и текущий count
        summary_count = neighbors_df['count'].sum() + row['count']
        
        if neighbors_df.empty or summary_count < min_neigbors_count_sum:
            return pd.Series([row['mean_area_price'], 0, 0, row['q1_area_price'], row['q3_area_price'], summary_count])
        
        total_count = row['count'] + neighbors_df['log_count'].sum()
        
        mean_price = (row['mean_area_price'] * row['count'] + neighbors_df['log_price'].sum()) / total_count
        q1_price = (row['q1_area_price'] * row['count'] + neighbors_df['log_q1_price'].sum()) / total_count
        q3_price = (row['q3_area_price'] * row['count'] + neighbors_df['log_q3_price'].sum()) / total_count
        
        return pd.Series([mean_price, q1_price, q3_price, row['q1_area_price'], row['q3_area_price'], summary_count])

    # Применяем вычисления к каждому ряду
    result = df.apply(calculate_for_row, axis=1)
    
    # Присваиваем результаты в соответствующие колонки
    df[['mean_price_neighbors', 'q1_price', 'q3_price', 'q1_area_price_new', 'q3_area_price_new', 'summary_count_current_plus_neighbors']] = result
    
    return df

def get_mean_price(ddf, log_base, min_neigbors_count_sum, ddf_all):
    # Применяем функцию к каждому разделу данных
    ddf = ddf.map_partitions(
        calculate_mean_price_with_h3_neighbors, 
        log_base=log_base, 
        min_neigbors_count_sum=min_neigbors_count_sum,
        df_all=ddf_all
    )
    return ddf

# Преобразуем в Dask DataFrame с более оптимальным числом партиций
ddf_with_missing = dd.from_pandas(df_with_missing, npartitions=10)

# Запуск с отображением прогресса
with ProgressBar():
    ddf_grouped = get_mean_price(ddf_with_missing, log_base, min_neigbors_count_sum, ddf_with_missing)
    # Выполняем вычисления (при необходимости)
    ddf_grouped = ddf_grouped.compute()


In [ ]:
ddf_with_missing = ddf_grouped
ddf_with_missing = ddf_with_missing[ddf_with_missing['mean_price_neighbors'] != 0]
ddf_with_missing.iloc[:, 2:-1] = ddf_with_missing.iloc[:, 2:-1].round()
ddf_with_missing = ddf_with_missing[ddf_with_missing['summary_count_current_plus_neighbors'] > 3]
ddf_with_missing = ddf_with_missing[['hex_7','area','q1_price','q3_price','mean_price_neighbors','summary_count_current_plus_neighbors']]
ddf_with_missing

In [ ]:
def h3_to_polygon(hex_id):
    geojson = h3.h3_to_geo_boundary(hex_id, geo_json=True)
    return Polygon(geojson)

# Преобразование 'hex_7' в геометрию
ddf_with_missing['geometry'] = ddf_with_missing['hex_7'].apply(h3_to_polygon)

# Создание GeoDataFrame
geo_df = gpd.GeoDataFrame(ddf_with_missing, geometry='geometry')

# Установка проекции (замените на нужную вам)
geo_df.crs = 'epsg:4326' 

# Сохранение в GPKG
geo_df.to_file("1608024_all_country_doma_dask.gpkg", driver="GPKG")